# Notebook : Évaluation du modèle sur différentes périodes avec Prometheus (Mensuelle)
Ce notebook charge un modèle de machine learning sauvegardé, évalue les performances 
du modèle mois par mois et enregistre les métriques dans Prometheus. A la fin, travaille avec le modèle au jour.

In [1]:


## Importation des bibliothèques et configuration Prometheus

import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from prometheus_client import Gauge, start_http_server
import mlflow
# serveur HTTP Prometheus sur le port 8000
start_http_server(8000)

# Configuration des métriques Prometheus
accuracy_gauge = Gauge('model_accuracy', 'Model Accuracy', ['month'])
f1_gauge = Gauge('model_f1_score', 'Model F1 Score', ['month'])
precision_gauge = Gauge('model_precision', 'Model Precision', ['month'])
recall_gauge = Gauge('model_recall', 'Model Recall', ['month'])
loss_gauge = Gauge('model_loss', 'Model Loss', ['month'])

# Statistiques descriptives pour les colonnes numériques
mean_gauge = Gauge('numeric_column_mean', 'Mean of numeric column', ['column', 'month'])
std_gauge = Gauge('numeric_column_std', 'Std of numeric column', ['column', 'month'])
min_gauge = Gauge('numeric_column_min', 'Min of numeric column', ['column', 'month'])
max_gauge = Gauge('numeric_column_max', 'Max of numeric column', ['column', 'month'])


### Artefacts


In [2]:


# Artefacts stockés (Modèles et données)
def load_mlflow_artifact(artifact_path):
    local_path = mlflow.artifacts.download_artifacts(artifact_path)
    with open(local_path, 'rb') as f:
        return joblib.load(f)

# Modèles et les artefacts
X_reduced = load_mlflow_artifact("mlflow_artifacts/X_reduced.pkl")
y = load_mlflow_artifact("mlflow_artifacts/y.pkl")
model_path = "mlflow_artifacts/bow_svd_model.h5"

# Modèle depuis MLflow
model = tf.keras.models.load_model(model_path)

# Compile le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Séparation des données par mois


In [3]:
# Divise les données en périodes mensuelles

df = pd.read_csv('db/cleaned_data_sample.csv')
df['CreationDate'] = pd.to_datetime(df['CreationDate'])
df['month'] = df['CreationDate'].dt.to_period('M')  # Ajout d'une colonne "mois"

# Obtient la liste des mois uniques dans les données
unique_months = df['month'].unique()

# Obtient les données d'un mois spécifique
def get_data_for_month(df, X_reduced, y, month):
    df_month = df[df['month'] == month]
    X_month = X_reduced[:len(df_month)]
    y_month = y[:len(df_month)]
    return X_month, y_month, df_month

## Statistiques descriptives et enregistrement des métriques dans Prometheus


In [4]:



def log_descriptive_statistics(df, month):
    """Calcule et enregistre les statistiques descriptives des colonnes numériques dans Prometheus."""
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    descriptive_stats = df[numeric_columns].describe()

    for col in numeric_columns:
        mean_gauge.labels(column=col, month=str(month)).set(descriptive_stats.loc['mean', col])
        std_gauge.labels(column=col, month=str(month)).set(descriptive_stats.loc['std', col])
        min_gauge.labels(column=col, month=str(month)).set(descriptive_stats.loc['min', col])
        max_gauge.labels(column=col, month=str(month)).set(descriptive_stats.loc['max', col])

    print(f"Statistiques descriptives pour {month} enregistrées dans Prometheus.")




## Évaluation du modèle sans réentraînement (Data Drift et Model Drift)

In [5]:


def evaluate_model(model, X_test, y_test, month):
    """Évalue les performances du modèle et enregistre les métriques dans Prometheus."""
    # Prédictions du modèle
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype("int32")
    
    # Calcul des métriques
    f1 = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')

    # Enregistrement des métriques dans Prometheus
    accuracy_gauge.labels(month=str(month)).set(accuracy)
    f1_gauge.labels(month=str(month)).set(f1)
    precision_gauge.labels(month=str(month)).set(precision)
    recall_gauge.labels(month=str(month)).set(recall)
    loss_gauge.labels(month=str(month)).set(loss)

    print(f"{month} - Loss: {loss}, Accuracy: {accuracy}, F1 Score: {f1}")

## Évaluation et statistiques mensuelles
Boucle sur chaque mois, calcule les statistiques descriptives et évalue le modèle

In [6]:
for month in unique_months:
    # Données pour le mois actuel
    X_month, y_month, df_month = get_data_for_month(df, X_reduced, y, month)

    # Statistiques descriptives pour ce mois
    log_descriptive_statistics(df_month, month)

    # Evaluation du modèle pour ce mois
    evaluate_model(model, X_month, y_month, month)

# Résultats
print("Évaluation mensuelle du modèle enregistrée dans Prometheus.")


Statistiques descriptives pour 2023-04 enregistrées dans Prometheus.
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2023-04 - Loss: 0.11331631243228912, Accuracy: 0.612130880355835, F1 Score: 0.4832492579951344
Statistiques descriptives pour 2023-10 enregistrées dans Prometheus.
 1/34 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2023-10 - Loss: 0.11111187189817429, Accuracy: 0.612994372844696, F1 Score: 0.4857132792692833
Statistiques descriptives pour 2024-04 enregistrées dans Prometheus.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step
2024-04 - Loss: 0.11695609241724014, Accuracy: 0.5615763664245605, F1 Score: 0.43992586971288794
Statistiques descriptives pour 2023-02 enregistrées dans Prometheus.


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step
2023-02 - Loss: 0.11363290995359421, Accuracy: 0.6114063262939453, F1 Score: 0.4813414372796587
Statistiques descriptives pour 2023-07 enregistrées dans Prometheus.
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step
2023-07 - Loss: 0.11282694339752197, Accuracy: 0.6106557250022888, F1 Score: 0.48093500165362696


...

c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step
2024-05 - Loss: 0.1151018887758255, Accuracy: 0.5849056839942932, F1 Score: 0.4407756132756133
Statistiques descriptives pour 2024-09 enregistrées dans Prometheus.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2024-09 - Loss: 0.11718586832284927, Accuracy: 0.6190476417541504, F1 Score: 0.18846153846153843
Statistiques descriptives pour 2024-08 enregistrées dans Prometheus.


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
2024-08 - Loss: 0.11250752955675125, Accuracy: 0.6578947305679321, F1 Score: 0.29666666666666675
Statistiques descriptives pour 2024-10 enregistrées dans Prometheus.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2024-10 - Loss: 0.22167757153511047, Accuracy: 0.0, F1 Score: 0.0
Évaluation mensuelle du modèle enregistrée dans Prometheus.


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

In [7]:
def evaluate_model_and_log_metrics(model, X_test, y_test):
    """Évalue les performances du modèle et enregistre les métriques dans Prometheus."""
    # Prédiction et évaluation du modèle
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype("int32")
    
    # Calcul des métriques
    f1 = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    accuracy = accuracy_score(y_test, y_pred)

    # Enregistrement des métriques dans Prometheus avec le label "global" pour indiquer que c'est une évaluation générale
    accuracy_gauge.labels(month="global").set(accuracy)
    f1_gauge.labels(month="global").set(f1)
    precision_gauge.labels(month="global").set(precision)
    recall_gauge.labels(month="global").set(recall)
    loss_gauge.labels(month="global").set(loss)

    print(f"Loss: {loss}, Accuracy: {accuracy}, F1 Score: {f1}, Precision: {precision}, Recall: {recall}")


508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step


ValueError: gauge metric is missing label values